<a href="https://colab.research.google.com/github/shoaibimt/CancerAI/blob/main/file_8_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import pickle
import numpy as np
!pip install padelpy
from padelpy import padeldescriptor
import json
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 52.3 MB/s eta 0:00:00


In [3]:
! unzip /content/drive/MyDrive/inhibitor_prediction/bcr_abl/chembl_data_retrieve/testing_folder/fingerprints_xml.zip

Archive:  /content/drive/MyDrive/inhibitor_prediction/bcr_abl/chembl_data_retrieve/testing_folder/fingerprints_xml.zip
  inflating: AtomPairs2DFingerprintCount.xml  
  inflating: AtomPairs2DFingerprinter.xml  
  inflating: EStateFingerprinter.xml  
  inflating: ExtendedFingerprinter.xml  
  inflating: Fingerprinter.xml       
  inflating: GraphOnlyFingerprinter.xml  
  inflating: KlekotaRothFingerprintCount.xml  
  inflating: KlekotaRothFingerprinter.xml  
  inflating: MACCSFingerprinter.xml  
  inflating: PubchemFingerprinter.xml  
  inflating: SubstructureFingerprintCount.xml  
  inflating: SubstructureFingerprinter.xml  


In [4]:
import glob
xml_files = glob.glob("*.xml")
xml_files.sort()
xml_files

['AtomPairs2DFingerprintCount.xml',
 'AtomPairs2DFingerprinter.xml',
 'EStateFingerprinter.xml',
 'ExtendedFingerprinter.xml',
 'Fingerprinter.xml',
 'GraphOnlyFingerprinter.xml',
 'KlekotaRothFingerprintCount.xml',
 'KlekotaRothFingerprinter.xml',
 'MACCSFingerprinter.xml',
 'PubchemFingerprinter.xml',
 'SubstructureFingerprintCount.xml',
 'SubstructureFingerprinter.xml']

In [5]:
Finp_list = ['AtomPairs2DCount',
 'AtomPairs2D',
 'EState',
 'CDKextended',
 'CDK',
 'CDKgraphonly',
 'KlekotaRothCount',
 'KlekotaRoth',
 'MACCS',
 'PubChem',
 'SubstructureCount',
 'Substructure']

In [6]:
# Load the best ML model
model_file = '/content/drive/MyDrive/inhibitor_prediction/bcr_abl/chembl_data_retrieve/ML_model_training/RandomForest_maxdepth10_nestimators200.bin'

In [7]:
model_file

'/content/drive/MyDrive/inhibitor_prediction/bcr_abl/chembl_data_retrieve/ML_model_training/RandomForest_maxdepth10_nestimators200.bin'

In [8]:
with open(model_file, 'rb') as f_in:
    model = pickle.load(f_in)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Below is the compound that I want it to test. Compound detail must be submitted in SMILES format and any arbitrary name can be given (name is just given for reference).

In [9]:
molecule = {'Cc1cc(Nc2ncc(C(=O)Nc3c(C)cccc3Cl)s2)nc(C)n1': 'CHEMBL364623'}

In [10]:
molecule

{'Cc1cc(Nc2ncc(C(=O)Nc3c(C)cccc3Cl)s2)nc(C)n1': 'CHEMBL364623'}

In [11]:
 molecule_df= pd.DataFrame(list(molecule.items()),columns=['smile','chembl_id'] )


In [12]:
molecule_df

,smile,chembl_id
0,Cc1cc(Nc2ncc(C(=O)Nc3c(C)cccc3Cl)s2)nc(C)n1,CHEMBL364623


In [13]:
mol_smi = molecule_df.copy(deep=True)
mol_smi

,smile,chembl_id
0,Cc1cc(Nc2ncc(C(=O)Nc3c(C)cccc3Cl)s2)nc(C)n1,CHEMBL364623


In [14]:
# Export .smi file to be used by padelpy
mol_smi.to_csv('/content/drive/MyDrive/inhibitor_prediction/bcr_abl/chembl_data_retrieve/testing_folder/molecule_test.smi', sep='\t', index=False, header=False)

In [15]:
mol_smi

,smile,chembl_id
0,Cc1cc(Nc2ncc(C(=O)Nc3c(C)cccc3Cl)s2)nc(C)n1,CHEMBL364623


In [16]:
fp = dict(zip(Finp_list, xml_files))
fp

{'AtomPairs2DCount': 'AtomPairs2DFingerprintCount.xml',
 'AtomPairs2D': 'AtomPairs2DFingerprinter.xml',
 'EState': 'EStateFingerprinter.xml',
 'CDKextended': 'ExtendedFingerprinter.xml',
 'CDK': 'Fingerprinter.xml',
 'CDKgraphonly': 'GraphOnlyFingerprinter.xml',
 'KlekotaRothCount': 'KlekotaRothFingerprintCount.xml',
 'KlekotaRoth': 'KlekotaRothFingerprinter.xml',
 'MACCS': 'MACCSFingerprinter.xml',
 'PubChem': 'PubchemFingerprinter.xml',
 'SubstructureCount': 'SubstructureFingerprintCount.xml',
 'Substructure': 'SubstructureFingerprinter.xml'}

In [17]:
fp['CDK']

'Fingerprinter.xml'

In [18]:
# Load .smi file and calculate CDK fingerprint from the molecule to be tested
fingerprint_outfile = 'CDK_example_testing.csv'

In [19]:
fingerprint_descriptorfile = 'Fingerprinter.xml'

In [20]:
import padelpy
from padelpy import padeldescriptor
fingerprint = 'CDK'
fingerprint_output_file = ''.join([fingerprint,'.csv']) #CDK.csv
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/content/drive/MyDrive/inhibitor_prediction/bcr_abl/chembl_data_retrieve/testing_folder/molecule_test.smi',
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml',
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

In [21]:
descriptors = pd.read_csv(fingerprint_output_file)
descriptors

,Name,FP1,FP2,FP3,FP4,FP5,FP6,FP7,FP8,FP9,...,FP1015,FP1016,FP1017,FP1018,FP1019,FP1020,FP1021,FP1022,FP1023,FP1024
0,CHEMBL364623,1,1,0,0,0,0,0,0,0,...,0,1,1,0,0,0,1,1,0,0


In [22]:
descriptors.to_csv('cdk_testing.csv', index= False)

In [23]:
cp cdk_testing.csv '/content/drive/MyDrive/inhibitor_prediction/bcr_abl/chembl_data_retrieve/testing_folder/'

In [24]:
 # Load CDK fingerprint results and names of the low variance features of CDK obtained in training
cdk_mol_df = pd.read_csv("cdk_testing.csv")

In [25]:
cdk_mol_df

,Name,FP1,FP2,FP3,FP4,FP5,FP6,FP7,FP8,FP9,...,FP1015,FP1016,FP1017,FP1018,FP1019,FP1020,FP1021,FP1022,FP1023,FP1024
0,CHEMBL364623,1,1,0,0,0,0,0,0,0,...,0,1,1,0,0,0,1,1,0,0


In [26]:
low_var_feat_names = pd.read_csv('/content/drive/MyDrive/inhibitor_prediction/bcr_abl/chembl_data_retrieve/testing_folder/cdk_low_var.csv')
low_var_feat_names

,FP2,FP3,FP7,FP9,FP12,FP14,FP15,FP17,FP22,FP23,...,FP1005,FP1006,FP1008,FP1009,FP1011,FP1012,FP1016,FP1017,FP1019,FP1023
0,0,0,0,0,0,1,1,1,1,0,...,1,0,0,1,1,0,1,0,1,1
1,0,0,0,0,0,1,1,1,1,0,...,1,0,0,1,1,0,1,0,1,1
2,0,0,0,0,0,1,1,1,1,1,...,1,0,0,1,1,0,1,1,1,1
3,0,0,0,0,0,1,1,1,1,0,...,1,0,1,1,1,0,0,0,1,1
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,1,0,1,0,0,1,0,1,0,0,...,0,0,1,1,1,1,1,0,1,0
394,0,0,0,0,0,1,0,1,0,0,...,1,0,1,1,0,1,1,1,1,0
395,0,0,0,0,0,1,0,1,0,0,...,0,0,1,1,0,1,1,0,1,0
396,0,0,0,0,0,1,0,1,0,0,...,1,0,1,1,0,1,1,1,1,0


In [ ]:
#Descriptor_Name= low_var_feat_names.columns[select.get_support(indices=True)]
col= list(low_var_feat_names.columns.values)
col

In [28]:
# Filter low variance features of CDK fingerprint from the molecule to be tested
cdk_low_var = cdk_mol_df.drop('Name', axis=1)
#cdk_low_var = cdk_mol_df[low_var_feat_names["col"].values]
cdk_low_var

,FP1,FP2,FP3,FP4,FP5,FP6,FP7,FP8,FP9,FP10,...,FP1015,FP1016,FP1017,FP1018,FP1019,FP1020,FP1021,FP1022,FP1023,FP1024
0,1,1,0,0,0,0,0,0,0,1,...,0,1,1,0,0,0,1,1,0,0


In [29]:
cdk_low_var1 = cdk_low_var.filter(col)
cdk_low_var1

,FP2,FP3,FP7,FP9,FP12,FP14,FP15,FP17,FP22,FP23,...,FP1005,FP1006,FP1008,FP1009,FP1011,FP1012,FP1016,FP1017,FP1019,FP1023
0,1,0,0,0,1,0,0,1,0,0,...,1,1,0,1,1,1,1,1,0,0


In [30]:
# Make predictions with the best model
y_pred = model.predict_proba(cdk_low_var1)[0, 1]
active = y_pred >= 0.5
active

True

In [31]:
# Save results
result = {
        'Probability of this compound being active against BCR-ABL is ': float(y_pred),
        'Active': bool(active)
    }

In [32]:
result

{'Probability of this compound being active against BCR-ABL is ': 0.72,
 'Active': True}

In [36]:
# Convert results to a json file for posting them into a web service
def convert_to_json(result):
    """
    Converts a result object to a JSON string.

    Args:
        result: The result object to convert.

    Returns:
        A JSON string representing the result object.
    """

    return jsonify(result)

In [ ]:
# Run the app in the main (this code not needed here)
"""if __name__ == "__main__":
    app.run(debug=True, host='0.0.0.0', port=9696)"""